<a href="https://colab.research.google.com/github/gabordun/Python_workshops/blob/master/import_from_web3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#import requests
from urllib import request
import requests
import time

basic=request.urlopen("https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions")
soup=BeautifulSoup(basic, 'html.parser')


#soup.table.find_all('tr')
#table = {'Hamilton' :[],'Vettel': [], 'Schumacher': []}

x = soup.table.find_all('td')
rowcounter = 0
for item in x:
  rowcounter += 1
y = soup.table.find_all('tr')
colcounter = 0
for item in x:
  colcounter += 1

print(rowcounter)
print(colcounter)

for i in range(0,rowcounter):
 print(get_text(x[i])


# print("--")

#for i in range(0,colcounter):
#  print(get_text(y[i])
# print("--")

#for tr in soup.find_all('tr'):
#   print(tr.th)
#   print(tr.td)
#   print("--")

#for item in soup.find_all('tr'):
# if item.th == '<a href="/wiki/1967_Formula_One_season" title="1967 Formula One season">1967</a>  <a href="/wiki/Denny_Hulme" title="Denny Hulme">D. Hulme</a><br/>':
#   table[item.th].append(item.td.text.replace('\n',''))
# if item.th == 'Vettel':
#  table[item.th.text].append(item.td.text)
# if item.th == 'Schumacher':
#  table[item.th.text].append(item.td.text)

#df = pd.DataFrame( data = table)
#df

#table

SyntaxError: ignored

In [19]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#import requests
from urllib import request
import requests
import time

url = "https://index.hu/kultur/zene/2020/01/03/legjobb_magyar_lemezek_2019/"
basic=request.urlopen(url)
soup=BeautifulSoup(basic, 'lxml')

#for item in soup.find_all('td'):
print(soup.get_text())
print("--")

tables = pd.read_html(url)

tables


#soup.table.find_all('tr')
#table = {'Hamilton' :[],'Vettel': [], 'Schumacher': []}

#x = soup.table_find_all()
#rowcounter = 0
#for item in x:
#  rowcounter += 1
#y = soup.table('tr')
#colcounter = 0
#for item in x:
#  colcounter += 1

#print(rowcounter)
#print(colcounter)

#for i in range(0,rowcounter):
#  print(y[i].get_text())

#for i in range(0,rowcounter):
# print(get_text(x[i])



Index - Kultúr - Ezeket a magyar lemezeket szerettük 2019-ben
















 



 

 














var _sf_startpt=(new Date()).getTime()
 























    (function(h,o,t,j,a,r){
        h.hj=h.hj||function(){(h.hj.q=h.hj.q||[]).push(arguments)};
        h._hjSettings={hjid:1219586,hjsv:6};
        a=o.getElementsByTagName('head')[0];
        r=o.createElement('script');r.async=1;
        r.src=t+h._hjSettings.hjid+j+h._hjSettings.hjsv;
        a.appendChild(r);
    })(window,document,'https://static.hotjar.com/c/hotjar-','.js?sv=');
 
            var korhatar_roadblock = 1952078;
        
<!--//--><![CDATA[//><!--
        var _gaq = { push: function() {} };
        ( function( i, s, o, g, r, a, m ) {
            i[ 'GoogleAnalyticsObject' ] = r;
            i[ r ] = i[ r ] || function() {
                ( i[ r ].q = i[ r ].q || [] ).push( arguments )
            }, i[ r ].l = 1 * new Date();
            a = s.createElement( o ), m = s.getElementsByTagName( o )[ 0 ];


ValueError: ignored

In [0]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#import requests
from urllib import request
import requests
import time

url = "https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"
tables = pd.read_html(url)

counter = 0
for x in tables:
 counter += 1

counter

for i in range(0,counter):
 print(tables[i])

#print(df)

#df2=pd.DataFrame(data = tables[counter-1])

#df2

#tables[counter-1]

                                         Formula One
0  Current season 2020 Formula One World Champion...
1  Related articles History of Formula One Formul...
2  Lists Drivers (GP winnersPolesittersFastest la...
3   Records Drivers Constructors Engines Tyres Races
4  Organisations FIA FIA World Motor Sport Counci...
5                                                vte
    Season                  Driver  Age  ...       Clinched  Margin  % Margin
0     1950     Giuseppe Farina [1]   44  ...    Race 7 of 7       3    10.000
1     1951  Juan Manuel Fangio [2]   40  ...    Race 8 of 8       6    19.355
2    19521      Alberto Ascari [3]   34  ...    Race 6 of 8      12    33.333
3    19531      Alberto Ascari [3]   35  ...    Race 8 of 9     6.5    18.841
4     1954  Juan Manuel Fangio [2]   43  ...    Race 7 of 9  16.857    40.136
..     ...                     ...  ...  ...            ...     ...       ...
67    2016       Nico Rosberg [33]   31  ...  Race 21 of 21       5     1.299
68   

In [0]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#import requests
from urllib import request
import requests
import time

basic=requests.get("https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions")
soup=BeautifulSoup(basic.text,'html.parser')
     
table = {'Driver': [], 'Season':[] , 'Age':[] }
#marker = 0
for item in soup.table.find_all('tr'):
 if item.tr == 'Driver':
   table[item.tr.text].append(item.td.text.replace('\n',''))
 if item.tr == 'Season':
   table[item.tr.text].append(item.td.text)
 if item.tr == 'Age':
   table[item.tr.text].append(item.td.text)
df = pd.DataFrame(data = table)
df


,Driver,Season,Age


In [0]:
# read source html
import requests
r = requests.get("https://en.wikipedia.org/wiki/Formula_One")
doc = BeautifulSoup(r.text, "html.parser")

new_table = pd.DataFrame(columns=range(0,100), index = [1]) # I know the size 

row_marker = 0
for row in doc.find_all('tr'):
  column_marker = 0
  columns = row.find_all('td')
  for column in columns:
    new_table.iat[row_marker,column_marker] = column.get_text()
    column_marker += 1
    
new_table

AttributeError: ignored

In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df